## Deliverable 2. Create a Customer Travel Destinations Map.

In [26]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [27]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Garmsar,IR,35.2182,52.3409,52.30,29,0,4.07,clear sky
1,1,Kruisfontein,ZA,-34.0033,24.7314,60.89,95,98,5.41,overcast clouds
2,2,Tucuma,BR,-0.1833,-50.0500,73.69,96,100,3.24,moderate rain
3,3,Nicoya,CR,10.1483,-85.4520,83.55,58,100,6.96,overcast clouds
4,4,Ponta Do Sol,PT,32.6667,-17.1000,64.00,70,58,5.41,broken clouds


In [28]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


In [29]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Tucuma,BR,-0.1833,-50.0500,73.69,96,100,3.24,moderate rain
3,3,Nicoya,CR,10.1483,-85.4520,83.55,58,100,6.96,overcast clouds
12,12,Dwarka,IN,22.2394,68.9678,77.68,83,0,10.85,clear sky
14,14,Quatre Cocos,MU,-20.2078,57.7625,75.31,71,39,8.01,scattered clouds
18,18,Wagar,SD,16.1525,36.2032,77.18,54,2,9.71,clear sky
21,21,Bathsheba,BB,13.2167,-59.5167,77.61,69,40,17.27,scattered clouds
23,23,Cayenne,GF,4.9333,-52.3333,80.10,1,40,4.61,scattered clouds
28,28,Iralaya,HN,15.0000,-83.2333,78.55,80,85,11.43,overcast clouds
29,29,Hithadhoo,MV,-0.6000,73.0833,82.56,66,23,5.68,few clouds
30,30,Alindao,CF,5.0267,21.2088,74.28,58,96,3.91,overcast clouds


In [30]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [31]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No rows to drop but the code to do so is below:
clean_pref_citites_df = preferred_cities_df.dropna()
clean_pref_citites_df.isna().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [32]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_pref_citites_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Tucuma,BR,73.69,moderate rain,-0.1833,-50.0500,
3,Nicoya,CR,83.55,overcast clouds,10.1483,-85.4520,
12,Dwarka,IN,77.68,clear sky,22.2394,68.9678,
14,Quatre Cocos,MU,75.31,scattered clouds,-20.2078,57.7625,
18,Wagar,SD,77.18,clear sky,16.1525,36.2032,
21,Bathsheba,BB,77.61,scattered clouds,13.2167,-59.5167,
23,Cayenne,GF,80.10,scattered clouds,4.9333,-52.3333,
28,Iralaya,HN,78.55,overcast clouds,15.0000,-83.2333,
29,Hithadhoo,MV,82.56,few clouds,-0.6000,73.0833,
30,Alindao,CF,74.28,overcast clouds,5.0267,21.2088,


In [33]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    
   # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [48]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace('',np.nan,regex = True)
clean_hotel_df = hotel_df.dropna()

In [51]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [56]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [57]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))